In [11]:
import requests
import json
import os
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint
from config import api_key

pd.options.display.float_format = "{:,.5f}".format

allmovies_df = pd.read_csv("allmoviesdata.csv")
allmovies_df

,Movie ID,Rank,Title,Production Budget,Domestic Gross,Worldwide Gross,Main Genre,2nd Genre,3rd Genre,IMDB Rating,Studio,Year,IMDb ID,Director
0,0,48,Black Panther,"200,000,000.00000","700,059,566.00000","1,336,494,320.00000",Action,Adventure,Drama,7.40000,Walt Disney Pictures,2018,NaN,Ryan Coogler
1,1,5,Avengers: Infinity War,"300,000,000.00000","678,815,482.00000","2,048,359,754.00000",Action,Adventure,Sci-Fi,8.50000,Walt Disney Pictures,2018,NaN,"Anthony Russo, Joe Russo"
2,2,50,Incredibles 2,"200,000,000.00000","608,581,744.00000","1,242,805,359.00000",Animation,Action,Adventure,7.80000,Pixar,2018,NaN,Brad Bird
3,3,144,Jurassic World: Fallen Kingdom,"170,000,000.00000","417,719,760.00000","1,308,323,302.00000",Action,Adventure,Drama,6.20000,Universal Pictures,2018,NaN,J.A. Bayona
4,4,11,Solo: A Star Wars Story,"275,000,000.00000","213,767,512.00000","393,151,347.00000",Action,Adventure,Drama,7.00000,Walt Disney Pictures,2018,NaN,Ron Howard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,204,"4,918",Annie Hall,"4,000,000.00000","38,251,425.00000","38,251,425.00000",Romance,Drama,Comedy,8.10000,Metro-Goldwyn-Mayer,1977,tt0075686,Woody Allen
217,205,"2,387",The Omen,"25,000,000.00000","54,607,383.00000","119,295,584.00000",Thriller,Horror,Drama,7.60000,20th Century Fox,1976,tt0075005,Richard Donner
218,206,44,King Kong,"207,000,000.00000","218,080,025.00000","550,517,357.00000",Horror,Fantasy,Adventure,5.80000,Paramount Pictures,1976,tt0074751,John Guillermin
219,207,445,Midway,"100,000,000.00000","56,846,802.00000","126,815,030.00000",History,Drama,Action,6.70000,Universal Pictures,1976,tt0074899,Jack Smight


In [20]:
#Creating a dataframe of directors, their budget, and gross for thier movies
directormoviecount = allmovies_df.groupby(["Director"]).count()["Title"]
directorbudget = allmovies_df.groupby(["Director"]).mean()["Production Budget"]
directordomestic = allmovies_df.groupby(["Director"]).mean()["Domestic Gross"]
directorworldwide = allmovies_df.groupby(["Director"]).mean()["Worldwide Gross"]
directors_df = pd.DataFrame({"Number of Movies":directormoviecount,
                            "Average Budget": directorbudget,
                            "Average Domestic Gross": directordomestic,
                            "Average Worldwide Gross": directorworldwide})

#Dataframe of top directors based on the number of movies that they made
moviedirect_df = directors_df.sort_values("Number of Movies", ascending=False)
moviedirect_df = moviedirect_df.head(50)

budgdirect_df = directors_df.sort_values("Average Budget", ascending=False)
budgdirect_df = budgdirect_df.head(50)

,Number of Movies,Average Budget,Average Domestic Gross,Average Worldwide Gross
Director,,,,
Rob Marshall,1,"379,000,000.00000","241,071,802.00000","1,045,713,802.00000"
Joss Whedon,2,"295,000,000.00000","541,181,889.00000","1,455,208,595.00000"
"Roger Allers, Rob Minkoff",1,"260,000,000.00000","543,638,043.00000","1,647,733,638.00000"
"Nathan Greno, Byron Howard",1,"260,000,000.00000","200,821,936.00000","583,777,242.00000"
"Anthony Russo, Joe Russo",3,"240,000,000.00000","448,882,263.00000","1,304,887,076.33333"
Andrew Adamson,1,"225,000,000.00000","141,621,490.00000","417,341,288.00000"
Barry Sonnenfeld,1,"215,000,000.00000","179,020,854.00000","654,213,485.00000"
Colin Trevorrow,1,"215,000,000.00000","652,306,625.00000","1,669,963,641.00000"
Zack Snyder,4,"212,000,000.00000","265,261,236.50000","662,625,438.25000"
